In [30]:
import os
import glob
import re
import pandas as pd
from pyproj import Transformer
import mgrs

# Función para convertir coordenadas de formato "10kmE<easting>N<northing>" a MGRS
def convert_10km_to_mgrs(coord_str: str) -> str:
    # Extraer índices de easting y northing
    match = re.match(r"10kmE(\d+)N(\d+)", coord_str)
    if not match:
        raise ValueError("El formato debe ser '10kmE<easting>N<northing>'")
    easting_idx = int(match.group(1))
    northing_idx = int(match.group(2))

    # Convertir índices a metros
    easting = easting_idx * 10000
    northing = northing_idx * 10000

    # Transformar de ETRS89/LAEA (EPSG:3035) a geográficas (EPSG:4326)
    transformer = Transformer.from_crs("EPSG:3035", "EPSG:4326", always_xy=True)
    lon, lat = transformer.transform(easting, northing)

    # Convertir a MGRS con precisión de 1 km
    mgrs_converter = mgrs.MGRS()
    mgrs_code = mgrs_converter.toMGRS(lat, lon, MGRSPrecision=1)
    return mgrs_code

# Función para ajustar valores del tipo "30TVL33" o "30TVL3231"
def ajustar_valor(cadena: str) -> str:
    # Procesa solo si la cadena empieza por alguno de los prefijos deseados
    if not (cadena.startswith("30") or cadena.startswith("29") or cadena.startswith("28") or cadena.startswith("31")):
        return cadena
    
    # Patrón 1: formato "<prefijo><letras><2 dígitos>" -> se deja igual.
    patron1 = r"^((?:30|29|28|31)[A-Z]+)(\d{2})$"
    # Patrón 2: formato "<prefijo><letras><2 dígitos><2 dígitos>" -> se elimina los dos últimos dígitos.
    patron2 = r"^((?:30|29|28|31)[A-Z]+)(\d{2})(\d{2})$"
    
    if re.match(patron1, cadena):
        return cadena
    elif re.match(patron2, cadena):
        m = re.match(patron2, cadena)
        return m.group(1) + m.group(2)
    else:
        return cadena

# Ruta de la carpeta con los Excel
ruta_carpeta = r"C:\Users\Ordenador\Documents\GitHub\Hackathon-PNAV\Datos\Datos EIDOS\EIDOS_ubicacion"
# Patrón para encontrar archivos Excel (.xlsx)
patron_excel = os.path.join(ruta_carpeta, "*.xlsx")

dataframes = []

# Procesar cada archivo Excel en la carpeta
for archivo in glob.glob(patron_excel):
    # Ignorar archivos temporales que comienzan con "~$"
    if os.path.basename(archivo).startswith("~$"):
        continue

    df = pd.read_excel(archivo)
    
    # Procesar fila por fila la columna I (índice 8)
    for idx in df.index:
        valor = str(df.iloc[idx, 8])
        
        # Si el valor es de tipo "10km...", aplicar la conversión a MGRS
        if valor.startswith("10km"):
            try:
                nuevo_valor = convert_10km_to_mgrs(valor)
                df.iloc[idx, 8] = nuevo_valor
            except Exception as e:
                print(f"Error en archivo {archivo}, fila {idx} con valor '{valor}': {e}")
        # Si el valor empieza por 30, 29, 28 o 31, ajustar el valor quitando 2 dígitos finales en caso de tenerlos
        elif (valor.startswith("30") or valor.startswith("29") or 
              valor.startswith("28") or valor.startswith("31")):
            nuevo_valor = ajustar_valor(valor)
            df.iloc[idx, 8] = nuevo_valor

    # Agregar una columna para identificar el archivo origen (opcional)
    df['ArchivoOrigen'] = os.path.basename(archivo)
    
    dataframes.append(df)

# Concatenar todos los DataFrames en uno solo
df_final = pd.concat(dataframes, ignore_index=True)

# Guardar el DataFrame final en un nuevo Excel
ruta_salida = os.path.join(ruta_carpeta, "resultado_consolidado.xlsx")
df_final.to_excel(ruta_salida, index=False)

print(f"Proceso completado. Excel consolidado guardado en: {ruta_salida}")


Proceso completado. Excel consolidado guardado en: C:\Users\Ordenador\Documents\GitHub\Hackathon-PNAV\Datos\Datos EIDOS\EIDOS_ubicacion\resultado_consolidado.xlsx


In [ ]:
import pandas as pd
import os

# Carpeta donde están los archivos Excel
folder_path = r"C:\Users\Ordenador\Documents\GitHub\Hackathon-PNAV\Datos\Datos EIDOS\EIDOS_ubicacion"
output_folder = r"C:\Users\Ordenador\Documents\GitHub\Hackathon-PNAV\Datos\Datos EIDOS\EIDOS_ubicacion"
os.makedirs(output_folder, exist_ok=True)

# Columnas clave para considerar duplicados
key_columns = ["nombre_aceptado", "cuadr"]  # Ajusta según los nombres exactos de las columnas

# Procesar cada archivo Excel en la carpeta
for file in os.listdir(folder_path):
    if file.endswith(".xlsx") or file.endswith(".xls"):
        file_path = os.path.join(folder_path, file)
        df = pd.read_excel(file_path)
        
        # Ordenar, eliminar duplicados y volver a ordenar
        df_sorted = df.sort_values(by=key_columns)
        df_cleaned = df_sorted.drop_duplicates(subset=key_columns, keep="first")
        df_final = df_cleaned.sort_values(by=key_columns)
        
        # Guardar archivo limpio con nombre "result.xlsx"
        output_path = os.path.join(output_folder, "mismo_cud_sin_repetidos_.xlsx")
        
        # Si el archivo ya existe, intenta eliminarlo primero
        if os.path.exists(output_path):
            os.remove(output_path)
        
        df_final.to_excel(output_path, index=False)
        print(f"Procesado y guardado como: result.xlsx")

print("Todos los archivos han sido procesados y guardados en la carpeta de salida.")



KeyboardInterrupt: 